In [1]:
import pandas as pd
import DataLoading
import NN_classes
import NN_evaluation
import torch
import matplotlib.pyplot as plt
import  numpy as np

In [15]:
def calculate_lb_from_ts_out(ts_out,ex):
    b= "Network_Existing_Generation_Full"
    all_ts_out_ex = torch.concat((ts_out["train"][ex],ts_out["test"][ex],ts_out["val"][ex]))
    all_ts_out_benchmark = torch.concat((ts_out["train"][b],ts_out["test"][b],ts_out["val"][b]))
    return all_ts_out_benchmark-all_ts_out_ex
    

def find_xthbest_model_params_from_df(df_losses,loss_to_sort,xth_best=1):
    return df_losses.sort_values(by =loss_to_sort)[xth_best-1:xth_best]

def extract_model_params_from_row(row):
    model_type = row.Model_type.item()
    model_type = tuple(map(int, model_type.replace("(","").replace(")","").split(', ')))
    dor = row.Dor.item()
    lr = row.Lr.item()
    nb_e = row.Epochs.item()

    relu_out = row.Relu_out.item()
    np =row.Np.item()
    bs = row.Batch_size.item()
    alpha = row.alpha.item()
    MAE = row.MAE.item()
    
    return {"Model_type": model_type,"nb_e":nb_e,"lr":lr,"dor":dor,"np":np,"ro":relu_out,"bs":bs,"alpha":alpha,"MAE":MAE}
    

def get_lb_est_and_actual(m,ex,dfs_in,dfs_out): 
    negf = all_executions[0]
    ex_in_e = torch.nan_to_num(dfs_in[ex].to_numpy()/maxs["in"])
    ex_in_negf = torch.nan_to_num(dfs_in[negf].to_numpy()/maxs["in"])

    prediction_e = m(ex_in_e.float())[0].detach().numpy()
    prediction_negf = m(ex_in_negf.float())[0].detach().numpy()

    lb_est = prediction_negf- prediction_e
    lb_actual = calculate_lb_from_dfs_out(dfs_out,ex).to_numpy()
    return lb_est.flatten(),lb_actual.flatten()

def get_NN_estimates_from_dfs_in(m,ex,dfs_in):
    ex_in_e = torch.nan_to_num(dfs_in[ex].to_numpy()/maxs["in"])
    prediction_e = m(ex_in_e.float())[0].detach().numpy()
    return prediction_e.flatten()
def get_actual_from_dfs_out(ex,dfs_out):
    actual = dfs_out[execution].sum(axis=1).to_numpy()
    return actual

In [3]:
sc = "sc01"
period = "2030"
#folder = "../Data/RTS24_AC_12w_ext_o_dummy/"
folder = "../Data/3-bus_AC_12w_ext_o_dummy_LCOE"

all_executions = DataLoading.list_executions(folder=folder,per = period,sc=sc)
te_s = 0.3
val_s = 0.4
dfs_in, dfs_out, dfs_inter = DataLoading.load_data_ext_out(folder, all_executions, period, sc, ["PowerFlow"])
dfs_inter_j = DataLoading.join_frames_inter_layer(dfs_inter)
dfs_inter_j = DataLoading.trim_columns_to_common(dfs_inter_j)
# Convert to pytorch tensors
ts_in, ts_out, ts_inter = DataLoading.split_tr_val_te_ext_out(dfs_in, dfs_out, dfs_inter_j, all_executions, te_s, val_s)
d_ft_in, d_ft_out, d_ft_inter,maxs = DataLoading.concat_and_normalize_ext_out(ts_in, ts_out, ts_inter, all_executions)



input_f_sc01_Network_Existing_Generation_Full_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac3_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac3_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac3_2030.csv
8


In [4]:
sc = "sc01"
period = "2030"
folder = "../Data/3-bus_AC_fy_ext_o_dummy_LCOE/"
all_executions = DataLoading.list_executions(folder=folder,per = period,sc=sc)
dfs_in_fy = DataLoading.load_data_input_only(folder,all_executions,period,sc)

input_size = dfs_in["Network_Existing_Generation_Full"].shape[1]
inter_size = dfs_inter_j["Network_Existing_Generation_Full"].shape[1]

input_f_sc01_Network_Existing_Generation_Full_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_2_cac3_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_1_Node_3_cac3_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac1_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac2_2030.csv
8
input_f_sc01_Network_Line_In_Node_2_Node_3_cac3_2030.csv
8


In [5]:
# hyperloop_name = "RTS24_AC_12w_dummy_0.3_v0.4_PF_sa_rand_0_40"
hyperloop_name = "3-bus_AC_12w_dummy_0.3_v0.4_PF_LCOE_0_10"
df_losses = pd.read_csv(f"ResultsClusterRuns/Loss_results_csv/{hyperloop_name}.csv",index_col=0)
#df_losses = pd.read_csv(f"Loss_results_csv/{hyperloop_name}.csv",index_col=0)


In [21]:
f = df_losses.alpha ==0
# f = df_losses.Model_type == "(3, 2)"
loss_to_sort = "Te_l_t_mse"
xth_best = 2
row = NN_evaluation.find_xthbest_model_params_from_df(df_losses[f],loss_to_sort,xth_best)
m = NN_evaluation.create_model_and_load_state_from_row(row,input_size,inter_size,hyperloop_name,cluster_run=True)

[16, 24, 5, 3, 1]


In [22]:
df_all_exec = pd.DataFrame()
folder_exports = "3-bus_AC_fy_ext_o_dummy_LCOE"
for ex in all_executions:
    prediction = NN_evaluation.get_NN_estimates_from_dfs_in(m,ex,dfs_in_fy,maxs)
    #actual= get_actual(ex,dfs_out) 
    df_all_exec[ex] = prediction

mpd = model_params_dict = NN_evaluation.extract_model_params_from_row(row)
keys = ["Model_type","nb_e","lr","dor","np","ro","bs","alpha","MAE"]
name = f"Estimated_OC_fy_{mpd[keys[0]]}h_{mpd[keys[1]]}e_{mpd[keys[2]]}lr_{mpd[keys[3]]}dor_{mpd[keys[4]]}np_{mpd[keys[5]]}ro_{mpd[keys[6]]}bs_{mpd[keys[7]]}ill_{mpd[keys[8]]}MAE"
df_all_exec.to_csv(f"NN_estimate_exports/{folder_exports}/{name}.csv")

In [ ]:
ex = all_executions[0]

df_all_exec = pd.DataFrame()
for ex in all_executions:
    prediction = get_NN_estimates_from_dfs_in(m,ex,dfs_in)
    #actual= get_actual(ex,dfs_out) 
    df_all_exec[ex] = prediction

mpd = model_params_dict = extract_model_params_from_row(row)
keys = ["Model_type","nb_e","lr","dor","np","ro","bs","alpha","MAE"]
name = f"Estimated_OC_12w_{mpd[keys[0]]}h_{mpd[keys[1]]}e_{mpd[keys[2]]}lr_{mpd[keys[3]]}dor_{mpd[keys[4]]}np_{mpd[keys[5]]}ro_{mpd[keys[6]]}bs_{mpd[keys[7]]}ill_{mpd[keys[8]]}MAE"
df_all_exec.to_csv(f"NN_estimate_exports/{name}.csv")